# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('../input/data.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Marcel \n\nProust \n\n\nIn Search \nof Lost \nTime \n\nVolume \n\ni \n\n\nSV^KNN 5 \nWAY \n\n\n\n\nJN S EARCH OF \nLOST T'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([ 20,  94,  67,  96,  29,  98,  72,  13,  13,  27,  67,  12,  32,
       110,  15,  72,  13,  13,  13,  21,  81,  72,  66,  29,  94,  67,
        96,  38,  72,  13,  12,  53,  72,  73,  12, 110,  15,  72,  13,
        83, 113,   1,  29,  72,  13,  13,  22,  12,  98,  32,   1,  29,
        72,  13,  13, 113,  72,  13,  13,  13,  66,  22,  40,  65, 103,
       103,  72,  54,  72,  13,  75,  74,  48,  72,  13,  13,  13,  13,
        13, 111, 103,  72,  66,  72,  10,  74,  28,  37, 101,  72,   0,
        36,  72,  13,  73,   0,  66,  83,  72,  83])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1, 2]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    batch_size_total = batch_size*seq_length
    # Get the number of batches we can make
    n_batches = len(arr)//(batch_size_total)
    
    # Keep only enough characters to make full batches
    arr = arr[:batch_size_total*n_batches]
    
    #Reshape into batch_size rows
    arr = arr.reshape(batch_size,-1)
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:,n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:,:-1],y[:,-1] = x[:,1:], arr[:,n+seq_length]
        except IndexError:
            y[:,:-1],y[:,-1] = x[:,1:], arr[:,0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 20  94  67  96  29  98  72  13  13  27]
 [ 94 113  67  49  72  94 110  72  38  29]
 [ 72  37  94  25  15  94 113  81  72  74]
 [ 15  94  15 113  51  29  98 112  72  94]
 [ 99  32  15  72  38  12  52  29  51  29]
 [112  12  32  72  94  67  29  72  94 110]
 [110  38  12  25  72  94  81  97  72 105]
 [ 12  81  72  52  38  29  81  72  38  29]]

y
 [[ 94  67  96  29  98  72  13  13  27  67]
 [113  67  49  72  94 110  72  38  29  72]
 [ 37  94  25  15  94 113  81  72  74  62]
 [ 94  15 113  51  29  98 112  72  94  81]
 [ 32  15  72  38  12  52  29  51  29  67]
 [ 12  32  72  94  67  29  72  94 110  16]
 [ 38  12  25  72  94  81  97  72 105  12]
 [ 81  72  52  38  29  81  72  38  29  72]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # defining the layers of the model
        self.lstm = nn.LSTM(len(self.chars),n_hidden,n_layers,dropout=drop_prob,batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden,len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ##Getting the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## passing through the dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [14]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden= 512
n_layers= 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(117, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=117, bias=True)
)


### Set your training hyperparameters!

In [20]:
batch_size = 256
seq_length = 120
n_epochs =  25

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/25... Step: 10... Loss: 2.7027... Val Loss: 2.6824
Epoch: 1/25... Step: 20... Loss: 2.6177... Val Loss: 2.6227
Epoch: 1/25... Step: 30... Loss: 2.5845... Val Loss: 2.6054
Epoch: 1/25... Step: 40... Loss: 2.5659... Val Loss: 2.5750
Epoch: 1/25... Step: 50... Loss: 2.5402... Val Loss: 2.5535
Epoch: 1/25... Step: 60... Loss: 2.5080... Val Loss: 2.5313
Epoch: 1/25... Step: 70... Loss: 2.5019... Val Loss: 2.5111
Epoch: 1/25... Step: 80... Loss: 2.4924... Val Loss: 2.4885
Epoch: 1/25... Step: 90... Loss: 2.4727... Val Loss: 2.4664
Epoch: 1/25... Step: 100... Loss: 2.4444... Val Loss: 2.4445
Epoch: 1/25... Step: 110... Loss: 2.4233... Val Loss: 2.4268
Epoch: 1/25... Step: 120... Loss: 2.3961... Val Loss: 2.4005
Epoch: 1/25... Step: 130... Loss: 2.3715... Val Loss: 2.3815
Epoch: 1/25... Step: 140... Loss: 2.3392... Val Loss: 2.3610
Epoch: 1/25... Step: 150... Loss: 2.3326... Val Loss: 2.3391
Epoch: 1/25... Step: 160... Loss: 2.2963... Val Loss: 2.3131
Epoch: 2/25... Step: 170... Loss:

Epoch: 9/25... Step: 1350... Loss: 1.4380... Val Loss: 1.4302
Epoch: 9/25... Step: 1360... Loss: 1.4546... Val Loss: 1.4277
Epoch: 9/25... Step: 1370... Loss: 1.4576... Val Loss: 1.4272
Epoch: 9/25... Step: 1380... Loss: 1.4486... Val Loss: 1.4255
Epoch: 9/25... Step: 1390... Loss: 1.4505... Val Loss: 1.4259
Epoch: 9/25... Step: 1400... Loss: 1.4480... Val Loss: 1.4267
Epoch: 9/25... Step: 1410... Loss: 1.4066... Val Loss: 1.4191
Epoch: 9/25... Step: 1420... Loss: 1.4348... Val Loss: 1.4185
Epoch: 9/25... Step: 1430... Loss: 1.4377... Val Loss: 1.4271
Epoch: 9/25... Step: 1440... Loss: 1.4610... Val Loss: 1.4185
Epoch: 9/25... Step: 1450... Loss: 1.4354... Val Loss: 1.4134
Epoch: 9/25... Step: 1460... Loss: 1.4272... Val Loss: 1.4117
Epoch: 9/25... Step: 1470... Loss: 1.4343... Val Loss: 1.4090
Epoch: 9/25... Step: 1480... Loss: 1.4232... Val Loss: 1.4061
Epoch: 10/25... Step: 1490... Loss: 1.4156... Val Loss: 1.4041
Epoch: 10/25... Step: 1500... Loss: 1.4202... Val Loss: 1.4017
Epoch:

Epoch: 17/25... Step: 2660... Loss: 1.3145... Val Loss: 1.3040
Epoch: 17/25... Step: 2670... Loss: 1.2942... Val Loss: 1.3043
Epoch: 17/25... Step: 2680... Loss: 1.3024... Val Loss: 1.2992
Epoch: 17/25... Step: 2690... Loss: 1.3089... Val Loss: 1.3027
Epoch: 17/25... Step: 2700... Loss: 1.3021... Val Loss: 1.2974
Epoch: 17/25... Step: 2710... Loss: 1.3042... Val Loss: 1.3031
Epoch: 17/25... Step: 2720... Loss: 1.2904... Val Loss: 1.3048
Epoch: 17/25... Step: 2730... Loss: 1.2627... Val Loss: 1.3004
Epoch: 17/25... Step: 2740... Loss: 1.2888... Val Loss: 1.2999
Epoch: 17/25... Step: 2750... Loss: 1.3007... Val Loss: 1.3014
Epoch: 17/25... Step: 2760... Loss: 1.3117... Val Loss: 1.2949
Epoch: 17/25... Step: 2770... Loss: 1.3013... Val Loss: 1.2948
Epoch: 17/25... Step: 2780... Loss: 1.2928... Val Loss: 1.2945
Epoch: 17/25... Step: 2790... Loss: 1.3006... Val Loss: 1.2982
Epoch: 17/25... Step: 2800... Loss: 1.2918... Val Loss: 1.2991
Epoch: 18/25... Step: 2810... Loss: 1.2782... Val Loss:

Epoch: 25/25... Step: 3970... Loss: 1.2356... Val Loss: 1.2596
Epoch: 25/25... Step: 3980... Loss: 1.2465... Val Loss: 1.2539
Epoch: 25/25... Step: 3990... Loss: 1.2317... Val Loss: 1.2530
Epoch: 25/25... Step: 4000... Loss: 1.2420... Val Loss: 1.2484
Epoch: 25/25... Step: 4010... Loss: 1.2484... Val Loss: 1.2554
Epoch: 25/25... Step: 4020... Loss: 1.2487... Val Loss: 1.2502
Epoch: 25/25... Step: 4030... Loss: 1.2446... Val Loss: 1.2511
Epoch: 25/25... Step: 4040... Loss: 1.2342... Val Loss: 1.2494
Epoch: 25/25... Step: 4050... Loss: 1.2026... Val Loss: 1.2519
Epoch: 25/25... Step: 4060... Loss: 1.2318... Val Loss: 1.2489
Epoch: 25/25... Step: 4070... Loss: 1.2329... Val Loss: 1.2492
Epoch: 25/25... Step: 4080... Loss: 1.2505... Val Loss: 1.2479
Epoch: 25/25... Step: 4090... Loss: 1.2336... Val Loss: 1.2493
Epoch: 25/25... Step: 4100... Loss: 1.2288... Val Loss: 1.2542
Epoch: 25/25... Step: 4110... Loss: 1.2311... Val Loss: 1.2480
Epoch: 25/25... Step: 4120... Loss: 1.2289... Val Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [21]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [22]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [23]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [25]:
print(sample(net, 1000, prime='I am ', top_k=5))

I am often as if his
marked her activity of the distant mind, and see that she was the salary
and that it was to bring the stop in the window. He did not have
been those that is traced in the soldiers, the memery there seemed to him to
that through his father.

And as shoot on the cruelty, when her house stopped him with the
sense of a certain serious and stoppings and watching her feet with
a sentence where their hands and the position was now, and to
them hardly. But the price of the son of harrows, at the son of
whom he was continually forgetting there, and was in which he was
stringed into the dream.

"You can shall be already as able to mon horse, there's not," she
said. "All you have been at a latter, but I won't be in the sense
of the sound."

"That's not always."

"This most still have the children."

"Why do you know, they have as an intention of society," he said
angrily about her, and was an excitement so much stringling.

"Oh, you'll be to do at her? You've been in the posi

## Loading a checkpoint

In [26]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [27]:
# Sample using a loaded model
print(sample(loaded, 400, top_k=5, prime="Once he "))

Once he came to the
manner of her bedroom of her son. He was the same thing in the
classing of the party, as 
